In [ ]:
import os

dir_names = ['content2','content3','content4']
dir_path = '/content/'

for name in dir_names:
    path = os.path.join(dir_path, name)
    if os.path.exists(path):
        os.rmdir(path)
    os.mkdir(path)

In [ ]:
!pip install boto3

In [ ]:
#Upload by folder
import os
import time
from datetime import datetime, timedelta
import boto3
import threading

# Let's use S3
s3 = boto3.client(
    service_name='s3',
    aws_access_key_id='accessid',
    aws_secret_access_key='acesskey',
    endpoint_url='https://s3.tebi.io'
)

# Set the folder path
folder_path = '/content/content3'

bytes_transferred = 0

def show_progress():
    global start_time, bytes_transferred, total_bytes
    elapsed_time = max(time.time() - start_time, 0.001)
    speed = (bytes_transferred / (1024 ** 2)) / elapsed_time
    remaining_bytes = total_bytes - bytes_transferred

    if bytes_transferred == 0:
        estimated_remaining_time = "Unknown"
    else:
        estimated_remaining_time = timedelta(seconds=remaining_bytes / (bytes_transferred / elapsed_time))

    print(f'\rUploaded: {bytes_transferred / (1024 ** 3):.2f}/{total_bytes / (1024 ** 3):.2f} GB, '
          f'Speed: {speed:.2f} MB/s, '
          f'Estimated remaining time: {estimated_remaining_time}', end='')

    # Schedule the next progress update
    threading.Timer(1, show_progress).start()

# Custom progress callback function to display upload progress
def progress_callback(new_bytes_transferred):
    global bytes_transferred
    bytes_transferred += new_bytes_transferred

# Loop through all items in the folder
for item_name in os.listdir(folder_path):
    item_path = os.path.join(folder_path, item_name)

    # Check if the item is a file
    if os.path.isfile(item_path):
        print(f"Uploading {item_name}")

        # Upload each file to S3
        with open(item_path, 'rb') as data:
            file_size = os.path.getsize(item_path)
            total_bytes = file_size
            start_time = time.time()  # Store the start time of the upload for calculating speed
            show_progress()  # Start showing progress updates
            s3.upload_fileobj(data, 'bucketname', item_name, Callback=progress_callback)
            print(f'\nUploaded {item_name}')
    else:
        print(f"Skipping {item_name} (not a file)")

print('All files uploaded successfully!')